In [1]:
import pandas as pd

In [2]:
transactions = pd.read_csv('./data/transactions_train.csv', dtype={'article_id': str}, parse_dates=['t_dat'])
transactions = transactions[transactions['t_dat'] >= '2020-09-01']

In [3]:
submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
most_bought_items_per_user = transactions.groupby(["customer_id", "article_id"])[["article_id"]].count()
most_bought_items_per_user.columns = ["purchase_count"]
most_bought_items_per_user = most_bought_items_per_user.reset_index()
most_bought_items_per_user = most_bought_items_per_user.sort_values(["customer_id", "purchase_count"], ascending=False)
most_bought_items_per_user = most_bought_items_per_user.groupby("customer_id")["article_id"].apply(list)

In [5]:
most_bought_items = list(transactions['article_id'].value_counts().index[:12])

In [6]:
submission["prediction"] = submission["customer_id"].map(most_bought_items_per_user)
submission["prediction"] = submission["prediction"].apply(lambda x: x if isinstance(x, list) else [])
submission["prediction"] = submission["prediction"].apply(lambda x: x[:12] + most_bought_items[:12-len(x)])
submission["prediction"] = submission["prediction"].apply(lambda x: " ".join(x))
submission.to_csv('./submission/stat_submission_10.csv', index=False)